In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [ ]:
#filter the data in 1980
df_1980 = df[df['year'] == 1980]

# Summing up TP over all entries for 1980 for each unique location
df_1980_agg = df_1980.groupby(['latitude', 'longitude']).agg(TP_1980=('TP', 'sum')).reset_index()

# Merge this aggregated data back to the main DataFrame
# This ensures every entry in the main DataFrame has a TP_1980 value corresponding to its location
df = df.merge(df_1980_agg, on=['latitude', 'longitude'], how='left')

In [ ]:
# Calculate the total and specific exposures based on HDDs as an example
df['Exp_TP'] = df['TP'].fillna(0) * df['HDD']
df['Exp_AP'] = df['AP'] * df['HDD']
df['UCM_ij'] = df['TP_1980'] * df['HDD']  # Using TP from 1980
df['UG_ij'] = (df['TP'].fillna(0) - df['TP_1980']) * df['HDD']

# Initialize a list to hold the regression results
results = []

# Perform regressions for each location and each metric
for (lat, lon), group in df.groupby(['latitude', 'longitude']):
    X = sm.add_constant(group['year'])  # Adding a constant term to the predictors

    # Regression models for exposure due to total precipitation and vapor pressure
    model_total = sm.OLS(group['Exp_TP'], X).fit()
    model_specific = sm.OLS(group['Exp_AP'].fillna(0), X).fit()
    model_heat = sm.OLS(group['UCM_ij'], X).fit()
    model_pop = sm.OLS(group['UG_ij'], X).fit()

    # Calculate index for difference between population growth and heating effect normalized by total exposure
    beta_pop = model_pop.params['year']
    beta_heat = model_heat.params['year']
    index = (beta_pop - beta_heat) / abs(beta_pop) if beta_pop != 0 else 0  # Handle division by zero

    # Append results including all coefficients and the index
    results.append({
        'latitude': lat,
        'longitude': lon,
        'beta_exp_TP': model_total.params['year'],
        'beta_exp_AP': model_specific.params['year'],
        'beta_UCM': beta_heat,
        'beta_UG': beta_pop,
        'Index': index
    })

# Convert results to a DataFrame for further analysis or export
results_df = pd.DataFrame(results)